In [ ]:
import os
os.getcwd()
import pandas as pd
import numpy as np

In [ ]:
master=pd.read_csv("Master.csv",encoding='latin')
tags=pd.read_csv("Tags.csv")
roles=pd.read_csv("Roles_v2.csv")
tags=tags[['job_id','tag']]

In [ ]:
master_jobid=master.loc[master['job_id'].isin(tags['job_id']),][['job_id']]
master_jobid=pd.merge(master_jobid,roles,on='job_id',how='left')
master_jobid=pd.merge(master_jobid,tags,on='job_id',how='left')
master_jobid.dropna(inplace=True)

master_transpose=pd.pivot_table(data=master_jobid,index=['job_id','role'],columns='tag',aggfunc={'role':len})
master_transpose.fillna(0,inplace=True)
master_transpose.columns=master_transpose.columns.droplevel()

master_summ=pd.DataFrame({'Count':master_transpose.sum(axis=1)})
master_transpose.reset_index(inplace=True)
master_transpose=master_transpose.loc[master_transpose['role']=="Nursing Assistant"]

Tag_count=master_transpose.drop(labels=['job_id','role'],axis=1).sum(axis=0)
Tag_count=Tag_count.to_frame()
Tag_count.reset_index(inplace=True)
Tag_count.columns=['tag','Count']
Tag_count.sort_values('Count',ascending=False,inplace=True)

Tag_count['Cum_sum']=Tag_count.cumsum(axis=0)['Count']
Tag_count['Cum_per']=100*Tag_count['Cum_sum']/sum(Tag_count['Count'])
Tag_80=Tag_count[Tag_count['Cum_per']<=80]

tag_80=Tag_count[Tag_count['Cum_per']<=80]
tag_80['Percent']=tag_80['Count']/sum(tag_80['Count'])


In [ ]:
col_names=['job_id','role']+tag_80['tag'].tolist()
subset_trans=master_transpose[col_names]
subset_trans['Doc_sum']=subset_trans.sum(axis=1)
subset_trans=subset_trans[subset_trans['Doc_sum']>0]
subset_trans['Doc_sum']=subset_trans.sum(axis=1)
subset_trans=subset_trans[subset_trans['Doc_sum']>0]

new_df=tag_80[['tag','Percent']].set_index('tag')
new_df.to_dict()


In [ ]:
def tf_idf2(df):
    for i in tag_80['tag'].tolist():
        col_name=i+'tf_idf'
        tf=df[i]/df['Doc_sum']
        idf=np.log10(1/(new_df['Percent'][i]))
        df[col_name]=tf*idf
    return df

In [ ]:
tf_matrix=tf_idf2(subset_trans)
tf_matrix.to_csv("NA_Tfidf.csv")